In [1]:
import json

config = json.load(open("config.json", encoding="utf-8"))

this configuration value of the configuration of the britghway adapter needs the name of the brightway project we want to operate on.

In [19]:
config["adapters"][0]["config"]["bw_project"]

'ecoinvent_391'

Regionalizations are defined by hierarchical location names, which are described in lists of strings for each activity.
Setting the regions of brighway activities we have two possibilities. Activities that are included in the 'hierarchy' can include their location in teir config with field `enb_location`. Like so:


In [3]:
config["hierarchy"]["children"][0]["config"]["enb_location"]

['EU', 'ESP', 'CAT']

For all other activities, for which the functional units are not required,  we can specify regions lists in the field `set_node_regions`, which is a dictionary, where the keys are brightway activity codes and the values are lists of region names. (Note that it's required but under normal circumstances the case that brightway activities in one project have unique code, tho it's not strickty required for brightway, since it also uses activities database as identifier, which we omitt here, under the given assumption).

It is also important to note, that a region that is described as a string, must be on the same index on all specified lists it is included. For example here:

```
{
"activity_0": ["EU", "DK"],
"activity_1": ["EU", "SW"],
"activity_4": ["EU", "ES", "CAT"],
"activity_5": ["EU", "ES", "BAC"],
}
```  

but not

```
{
"activity_0": ["EU", "DK"],
"activity_1": ["ES", "CAT"],
"activity_2": ["EU", "ES", "BAC"],
}
```  

since "ES" (Spain) appears on index 1 for activity 1 but on index 2 for activity 2.


In [23]:
config["adapters"][0]["config"]["simple_regionalization"]["set_node_regions"]


{'<code>': ['ES', 'CAT']}

In [31]:
from enbios.bw2.brightway_experiment_adapter import BrightwayAdapter
from enbios import Experiment

exp = Experiment(config)
bw_adapter: BrightwayAdapter = exp.get_adapter_by_name("brightway-adapter")
bw_adapter.prepare_regionalization()

Excluding 0 filtered results
Excluding 104 filtered results
2024-06-03 17:58:54,702 - /home/ra/projects/enbios/enbios/bw2/brightway_experiment_adapter.py - WARNING - Some activities specified in 'set_node_regions' are not found: {'<code>'}


```
# is the regionalization should be applied (default: false)
run_regionalization: boolean

# the regions of which we want the regionalization impacts of. a list of strings, which must be included in the defined locations. Which levels of the regional are selected does not matter. (e.g. it could be ["EU", "DK", "CAT]) and those regions would be included in the results.
select_regions: list of regions
# As explained above, locations of activities not included in the hierarchy 
set_node_regions: dict of regions (as described above)
# all other activities get their 'enb_location' field deleted, which acts as a reset (default: false).
clear_all_other_node_regions: boolean
```
The locations that brightway stores in activities coming from Ecoinvent in the activities under the key: "location", is not considered for the enbios regionalization. 

When the `run_regionalization` is set to true, enbios will do modifications to the activities as they ar stored in brightway.
All nodes, which are defined in the hierarchy with an `enb_locations` will get that set during the node validation done by the brightway adapter. All other activities and the reset (clear_all_other_node_regions) are performed before the first scenario is run.


## Step by step changes that enbios applies to brightway activities
Enbios will search for all relevant activities and also set the key 'enb_location', which is not use by brightway for anything accordingly. When the LCI is calculated it splits the LCI matrix by regions, before applying the characterization, so that it can give regionalized impacts.  

In [4]:
import bw2data
# first select the project
bw2data.projects.set_current("ecoinvent_391")
bw2data.databases

Databases dictionary with 2 object(s):
	biosphere3
	ecoinvent_391_cutoff

In [8]:
# but we also select the right database before searching for activities
db = bw2data.Database("ecoinvent_391_cutoff")

In [25]:
# let's look at the specif of the activity in our hierarchy, as they are needed to pinpoint the activity we are looking for.
# Consider that in other examples, we simply used the activities code to identify activities. However, with a combination of name, location and unit, enbioes is able to filter candidates accordingly and works if the result is one single activity. 
activity_config = config["hierarchy"]["children"][0]["config"]
activity_config

{'name': 'heat and power co-generation, wood chips, 6667 kW, state-of-the-art 2014',
 'unit': 'kilowatt hour',
 'location': 'DK',
 'enb_location': ['EU', 'ESP', 'CAT'],
 'default_output': {'unit': 'kWh', 'magnitude': 1}}

In [27]:
found_activities = db.search(activity_config["name"], filter={"location":"DK"})
found_activities

Excluding 104 filtered results


['heat and power co-generation, wood chips, 6667 kW, state-of-the-art 2014' (megajoule, DK, None),
 'heat and power co-generation, wood chips, 6667 kW, state-of-the-art 2014' (kilowatt hour, DK, None)]

In [15]:
# the 2nd activity matches the unit (index 1). We can set its 'enb_location' and save it.

'kilowatt hour'

In [ ]:
found_activities[1]["enb_location"] = ["EU", "DK"]
found_activities[1].save()

In [ ]:
# Note that enbios, uses other brightway internal mechanisms to set all locations, which is much more performant that as described above. However it also possible to call the function in the adapter that makes these changes directly
